## Fourier series and Data encoding for QML

Besides from practical approach of QML, the authors of [this article](https://arxiv.org/pdf/2008.08605.pdf) has shown the data encoding can be encapsulated in [Fourier series](https://mathworld.wolfram.com/FourierSeries.html)[1]. The underlying assumption for such demonstration is the encoder is an unitary gate in the form of $e^{-ixH}$. H is some arbitary Hamiltonian of the system and x is the data to encode. This sums down to evolution of the state by 'x' amount, moreover, if we take H to be a Pauli gate then the evolution turns into rotation of the state. 

The authors then goes on to demonstrate the link between different terms in Fourier series and the elements of the Quantum model. A quick glimpse on what are the links are below.

The series is written as,<br></br> 
$f_{\theta}(x) = \Sigma_{\omega\in\Omega} c_\omega(\theta)e^{i\omega x}$

The Time evolution of quantum state in Schrodinger's picture becomes pretty handy here. The Hamiltonian operator is replaced with it's eigenvalues and multiplied with the respective eigenvector to evolve the eigenstate. The linear superposition of all such operations are deemed as the evolution of the entire quantum state. We can observe a similar fashion in the fourier series. The frequency spectrum of the Pauli gates is $\Omega \subset \R^N$, in natural setting this always turns out to be set of spectrum is set of Integers i.e. $\Omega \subset \Z^N$. The eigenvalues of the Pauli matrices are $1$ and $-1$, a repeating application of the gate say n-times yields n-rotations of $\pi/2$. So, the Fourier series can be reformulated as below.

$f_{\theta}(x) = \Sigma_{n\in\Omega} c_n(\theta)e^{in x}$

Now back to the main concerns of links, the frequency spectrum is solely determined by the data-encoding Hamiltonian (Pauli gates in this case). The rest of the circuit (Quantum model), including the trainable parameters, determines the co-efficients. This means a target state has a rigid spectrum and the rest of the parameters, like the co-efficients, inputs and so on, can be arbitrarily initiated.

### Goal of data encoding

The authors succinctly write their ultimate goal in the article, it's quoted below.

> we study the universality of quantum models. We show that for suﬃciently ﬂexible trainable circuit blocks there exists a quantum model which can realise any possible set of Fourier coeﬃcients. If, asymptotically, the accessible frequency spectrum is rich enough, then such models are universal function approximators. This follows from the fact that Fourier series with arbitrary coeﬃcients can approximate any square integrable function on a given interval.


Before they demonstrate the universality, their approach is to slowly level up the game. This notebook too, will follow in their footsteps. In an one-qubit system, the bareminimum process is to encode the data only once into the angle of a single qubit rotation. This leads to the function class such that the quantum models can only learn up to a simple sine function (or equivalently, a Fourier series with a single frequency). The encoding occurs either sequential or parallel application of the encoding gate(s).

An anaytical methods to generalise our upcoming computational approach will go long way. To ease the thought process a schematic from the paper is placed below.

<figure>
<img src="assets/images/qc-model-general.png" alt="Quantum Data encoding model" width="600"/>
<figcaption><b>Figure 1 - Quantum Data encoding model for a multi-qubit system</b></figcaption>
</figure>

The above schematic is general. For the one qubit system it can be reduced into a smaller form like following.

<figure>
<img src="assets/images/qc-model.png" alt="Quantum Data encoding model" width="500"/>
<figcaption><b>Figure 1 - Quantum Data encoding model for a single qubit system</b></figcaption>
</figure>


## References

1. Schuld, Maria, Ryan Sweke, and Johannes Jakob Meyer. "Effect of data encoding on the expressive power of variational quantum-machine-learning models." Physical Review A 103.3 (2021): 032430.